<a href="https://colab.research.google.com/github/g-nxn/Project/blob/main/MedianRentZip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests

#pf.set_option('display.max_column', None)

In [8]:
census_api_key = '9cded9acb805f2c4fd58851fd43e25751c051fd4'

In [9]:
variable = 'B25031_001E' # median rent all beds
year = '2020'


In [19]:
# read zip to tract data from HUD
hud_tract_url = 'https://raw.githubusercontent.com/g-nxn/Project/main/ZIP_TRACT_032021.csv'
_df_hud_tract = pd.read_csv(hud_tract_url)
# dataframe detail
print('Num of rows:', len(_df_hud_tract))
print('Num of columns:', len(_df_hud_tract.columns))
_df_hud_tract.head()


Num of rows: 172038
Num of columns: 8


,ZIP,TRACT,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,501,36103158607,HOLTSVILLE,NY,0.000000,1.000000,0.000000,1.000000
1,601,72001956600,ADJUNTAS,PR,0.172037,0.358722,0.368098,0.189057
2,601,72113071700,ADJUNTAS,PR,0.160256,0.199017,0.128834,0.161936
3,601,72001956800,ADJUNTAS,PR,0.014726,0.012285,0.042945,0.015295
4,601,72001956700,ADJUNTAS,PR,0.652980,0.429975,0.460123,0.633712


In [34]:
df_hud_tract = _df_hud_tract.copy()
df_hud_tract['ZIP'] = df_hud_tract.apply(lambda x: zip_code_to_str(x['ZIP']), axis=1)
df_hud_tract['TRACT'] = df_hud_tract.apply(lambda x: 
  '0' + str(x['TRACT']) if len(str(x['TRACT'])) == 10 else str(x['TRACT']), axis=1)
df_hud_tract['state_code'] = df_hud_tract.apply(lambda x: x['TRACT'][:2], axis=1)
df_hud_tract['county_code'] = df_hud_tract.apply(lambda x: x['TRACT'][2:5], axis=1)
df_hud_tract['tract_code'] = df_hud_tract.apply(lambda x: x['TRACT'][5:], axis=1)
df_hud_tract.head()

,ZIP,TRACT,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,state_code,county_code,tract_code
0,00501,36103158607,HOLTSVILLE,NY,0.000000,1.000000,0.000000,1.000000,36,103,158607
1,00601,72001956600,ADJUNTAS,PR,0.172037,0.358722,0.368098,0.189057,72,001,956600
2,00601,72113071700,ADJUNTAS,PR,0.160256,0.199017,0.128834,0.161936,72,113,071700
3,00601,72001956800,ADJUNTAS,PR,0.014726,0.012285,0.042945,0.015295,72,001,956800
4,00601,72001956700,ADJUNTAS,PR,0.652980,0.429975,0.460123,0.633712,72,001,956700


In [21]:
def zip_code_to_str(x):
  x_str = str(x).split('.')[0]
  if len(x_str) == 5:
    return x_str
  elif len(x_str) == 4:
    return '0' + x_str
  elif len(x_str) == 3:
    return '00' + x_str


In [46]:
# view sample zipcode
zipcode = '40205' #Jefferson County, Louisville, KY
df_zip_sample = df_hud_tract.loc[df_hud_tract['ZIP'] == zipcode]
df_zip_sample

,ZIP,TRACT,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,state_code,county_code,tract_code
72331,40205,21111011200,LOUISVILLE,KY,0.000000,0.005882,0.000000,0.000313,21,111,011200
72332,40205,21111009600,LOUISVILLE,KY,0.187038,0.101471,0.075775,0.174906,21,111,009600
72333,40205,21111008700,LOUISVILLE,KY,0.029289,0.083824,0.065442,0.034653,21,111,008700
72334,40205,21111013100,LOUISVILLE,KY,0.084750,0.030882,0.005741,0.076502,21,111,013100
72335,40205,21111009300,LOUISVILLE,KY,0.003383,0.016176,0.013777,0.004772,21,111,009300
72336,40205,21111009400,LOUISVILLE,KY,0.021188,0.002941,0.001148,0.018852,21,111,009400
72337,40205,21111010901,LOUISVILLE,KY,0.000000,0.001471,0.000000,0.000078,21,111,010901
72338,40205,21111008800,LOUISVILLE,KY,0.146176,0.108824,0.239954,0.150579,21,111,008800
72339,40205,21111010500,LOUISVILLE,KY,0.017627,0.179412,0.040184,0.027769,21,111,010500
72340,40205,21111009700,LOUISVILLE,KY,0.131042,0.045588,0.080367,0.123044,21,111,009700


In [47]:
# prepare parameters
tract_str = ','.join(df_zip_sample['tract_code'].tolist())
state = df_zip_sample['state_code'].iloc[0]
county = df_zip_sample['county_code'].iloc[0]
print('Year:', year)
print('Variable:', variable)
print('Tracts:',  tract_str)
print('State:', state)
print('County:', county)

Year: 2020
Variable: B25031_001E
Tracts: 011200,009600,008700,013100,009300,009400,010901,008800,010500,009700,008500,008900,008400
State: 21
County: 111


In [48]:
# get data from census
url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=tract:{2}&in=state:{3}%20county:{4}&key={5}'\
  .format(year, variable, tract_str, state, county, census_api_key)
response = requests.request("GET", url)
response.status_code
     

200

In [49]:
# read rent data
df_zip = pd.DataFrame(response.json()[1:], columns=response.json()[0])\
  .rename(columns={'B25031_001E': 'median_rent_all_bds'})
print('Num of rows:', len(df_zip))
df_zip.sort_values(by=['median_rent_all_bds'])

Num of rows: 11


,NAME,median_rent_all_bds,state,county,tract
5,"Census Tract 93, Jefferson County, Kentucky",1000,21,111,009300
4,"Census Tract 89, Jefferson County, Kentucky",1019,21,111,008900
0,"Census Tract 84, Jefferson County, Kentucky",1073,21,111,008400
8,"Census Tract 105, Jefferson County, Kentucky",1192,21,111,010500
10,"Census Tract 131, Jefferson County, Kentucky",1313,21,111,013100
7,"Census Tract 97, Jefferson County, Kentucky",1318,21,111,009700
9,"Census Tract 109.01, Jefferson County, Kentucky",790,21,111,010901
2,"Census Tract 87, Jefferson County, Kentucky",897,21,111,008700
6,"Census Tract 96, Jefferson County, Kentucky",900,21,111,009600
1,"Census Tract 85, Jefferson County, Kentucky",934,21,111,008500


In [50]:
print('For zipcode:{0} median rent in {1} is ${2}'.format(zipcode, year, df_zip['median_rent_all_bds'].median()))
     

For zipcode:40205 median rent in 2020 is $1000.0


Part two


Section #2 - Loops 🏙
Get data for multiple years for a given region

In [51]:
# list of years for Census API
year_list = ['2015', '2016', '2017', '2018', '2019', '2020']

In [52]:
# iterate through list of years
df_list = []
for year in year_list:
  # get median rent for the sample zipcode
  url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=tract:{2}&in=state:{3}%20county:{4}&key={5}'\
    .format(year, variable, tract_str, state, county, census_api_key)
  response = requests.request("GET", url)
  _df = pd.DataFrame(response.json()[1:], columns=response.json()[0])
  _df['year'] = year
  df_list.append(_df)

# combine responses into a single dataframe
df_census = pd.concat(df_list)\
  .rename(columns={'B25031_001E': 'median_rent_all_bds'})
df_census['median_rent_all_bds'] = df_census['median_rent_all_bds'].astype(int)
print('Num of rows:', len(df_census))
df_census.head()
     

Num of rows: 76


,NAME,median_rent_all_bds,state,county,tract,year
0,"Census Tract 97, Jefferson County, Kentucky",848,21,111,009700,2015
1,"Census Tract 85, Jefferson County, Kentucky",739,21,111,008500,2015
2,"Census Tract 131, Jefferson County, Kentucky",1250,21,111,013100,2015
3,"Census Tract 89, Jefferson County, Kentucky",711,21,111,008900,2015
4,"Census Tract 87, Jefferson County, Kentucky",914,21,111,008700,2015


In [53]:
# group by year
df_census_grp = df_census.groupby(['year'])['median_rent_all_bds'].median().reset_index()
df_census_grp

,year,median_rent_all_bds
0,2015,729.0
1,2016,794.0
2,2017,860.0
3,2018,859.0
4,2019,921.0
5,2020,1000.0


In [54]:
fig = px.line(df_census_grp, x='year', y='median_rent_all_bds', title='Median Rent by Year for {}'.format(zipcode))
fig.show()